In [7]:
from scipy.stats import poisson
import numpy as np
import warnings

# cdf of the truncated poisson distribution
def truncPoisCdf(y, mu, log=True):
    is_scalar = np.isscalar(y)  # check if y is scalar or array

    if is_scalar:
        y = np.array([y])  # typecast scalar to onedimensional array

    f_zero = poisson.pmf(0, mu) # f(0) untruncated density

    # general formula for lower trunc. distributions
    cdf_y = (poisson.cdf(y, mu) - poisson.cdf(0, mu)) / (1 - f_zero)
    # VERY IMPORTANT STEP: there might be numerical instabilities, which may lead to
    # poisson.cdf(x) < poisson.cdf(0)!!!!! this then leads to negative values of the cdf
    # or nans in the log version (np.log(neg number))
    cdf_y[cdf_y < 0] = 0 # set negative values to zero
    # set values to 0, if y <= 0 (y <=0 not allowed per definition of a 0 truncated count distribution)
    cdf_y[y <= 0] = 0

    # in case of log CDF
    if log:
        # ignore the 'RuntimeWarning: divide by zero encountered in log' warning (np.log(0))
        warnings.filterwarnings('ignore', category=RuntimeWarning)
        log_cdf_y = np.log(cdf_y) # general formula for lower trunc. distributions
        warnings.filterwarnings('default', category=RuntimeWarning)

        if is_scalar:
            return log_cdf_y[0]  # return scalar, if onedimensional array
        else:
            return log_cdf_y
    # normal CDF    
    else:
        if is_scalar:
            return cdf_y[0]
        else:
            return cdf_y

        
def log1mexp(x):
    if np.any((x < 0) & (~np.isnan(x))):
        raise ValueError("Inputs need to be non-negative!")
    return np.where(x <= np.log(2), np.log(-np.expm1(-x)), np.log1p(-np.exp(-x)))
        

#log.p	logical; if TRUE, probabilities p are given as log(p)    
def qpois_trunc(p, lam, log_p=False):

    # values of lam <= 0 are not allowed. return nan
    if lam <= 0:
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        return np.full(len(p), np.nan)

    # if f(0)=0 no truncation is needed
    if poisson.pmf(0, lam) == 0:
        return poisson.ppf(p, lam)
    else:
        # Convert p (quantile) to array if it's a scalar
        if not isinstance(p, (list, np.ndarray)):
            p = np.array([p])
        
        n = len(p) # number of quantiles

        # Set log-probabilities (lower tail)
        if log_p:
            logp = p
        else:
            logp = np.log(p)
        
        # Set output and deal with special cases (outputs NA and Inf)
        quantiles = np.full(n, np.nan)
        nna = ~np.isnan(logp)
        nlogp = logp[nna]
        if len(nlogp) == 0:
            return quantiles
        
        quantiles[nna] = np.full(len(nna), np.inf)
        if np.min(nlogp) >= 0:
            return quantiles

        # Set log-CDF vector
        lp_max = np.max(nlogp[nlogp < 0]) # find the highest log quantile to calculate
        p_max = np.exp(lp_max) # highest quantile

        # find an adequate upper limit, starting from the extreme conservative chebychev inequality
        upper = int(lam + np.sqrt(lam * np.exp(-log1mexp(-lp_max)))) #Chebychev inequality

        # if upper < 1000 there is an log(0)=-inf with warning -> ignore this warning
        warnings.filterwarnings('ignore', category=RuntimeWarning)

        while truncPoisCdf(upper-1000, lam, log=False) > p_max:
            upper = upper - 1000

        # after this section warnings are enabled again
        warnings.filterwarnings('default', category=RuntimeWarning)

        yarray = np.arange(1, int(upper)+1) # the y values for which the CDF is going to be calculated
        logcdf = truncPoisCdf(yarray, lam) # calculate log CDF (faster computation time)

        # Compute output
        for i in range(n): # for all quantiles   
            if nna[i]:     # that are not na
                if logp[i] < 0: # and logp<0 count the number of observations (y): cdf(y) < quantile
                    quantiles[i] = np.sum(logcdf < np.array(logp[i])) + 1 #+1 because 0 is truncated
        
        # Return output
        if len(quantiles) == 1:
            return quantiles[0] # if single quantile is handed over
        else:
            return quantiles

In [2]:
# calculation of quantiles
quantiles = np.arange(0.001, 0.9999, 0.001)
quantiles = np.round(quantiles, 3)

lam = 6

a = poisson.ppf(quantiles, lam)
b = qpois_trunc(quantiles, lam)

In [3]:
a

array([ 0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  1.,  1.,  1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [4]:
b

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
        1.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.,
        2.,  2.,  2.,  2.,  2.,  2.,  2.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.,
        3.,  3.,  3.,  3.,  3.,  3.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,
        4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.,  4

In [5]:
a == b

array([False, False,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False, False,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True, False, False,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,